Домашнее задание №0
====



In [1]:
# Импортируем нужные нам библиотеки
import pandas as pd
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn; seaborn.set()
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from sklearn.model_selection import train_test_split
import matplotlib.dates as mdates
import seaborn as sns
import matplotlib
# Use seaborn style defaults and set the default figure size
sns.set(rc={'figure.figsize':(15, 10)})
sns.set(font_scale=2)
sns.set_style("whitegrid")
from plotly.graph_objects import *
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

In [2]:
# Загружаем датасет, с которым мы будем работать
df = pd.read_csv('../datatest/bike-sharing.csv')

In [3]:
df.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,9.84,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,9.02,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,9.02,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,9.84,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,9.84,0.2879,0.75,0.0,0,1,1


Задание
====

* Выбрать самостоятельно уровень агрегации (день, неделя, месяц, год), а так же функцию агрегации (среднее, сумма, максимум). 
* Построить базовую наивную модель





Начну с рассматривание что какое поле означает с целью определить таргет и понять какие поля мне интересны, а какие можно исключить:
* instant: record index
- dteday : date
- season : season (1:winter, 2:spring, 3:summer, 4:fall)
- yr : year (0: 2011, 1:2012)
- mnth : month ( 1 to 12)
- hr : hour (0 to 23)
- holiday : weather day is holiday or not (extracted from [Web Link])
- weekday : day of the week
- workingday : if day is neither weekend nor holiday is 1, otherwise is 0.
- weathersit :
  - 1: Clear, Few clouds, Partly cloudy, Partly cloudy
  - 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
  - 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
  - 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog
- temp : Normalized temperature in Celsius. The values are derived via (t-t_min)/(t_max-t_min), t_min=-8, t_max=+39 (only in hourly scale)
- atemp: Normalized feeling temperature in Celsius. The values are derived via (t-t_min)/(t_max-t_min), t_min=-16, t_max=+50 (only in hourly scale)
- hum: Normalized humidity. The values are divided to 100 (max)
- windspeed: Normalized wind speed. The values are divided to 67 (max)
- casual: count of casual users
- registered: count of registered users
- cnt: count of total rental bikes including both casual and registered

С таргетом понятно - последнее поле. ну или может быть соотношение casual к registered. Но возьмем просто итог

In [4]:
df = df[['dteday','cnt']] # в рамках лабораторной работы интересует нас только дата и таргет, их и оставляем
df['dteday']=pd.to_datetime(df['dteday']) # превращаем "дату" в Дату
df.set_index('dteday',inplace=True) # Дату делаем индексом - теперь это можно называть временным рядом, готовым для работы
result_table = pd.DataFrame(columns=['Агрегация', 'Функция агрегации', 'MAE, lag_1', 'MAE for lag_2']) # сюда буду складывать результаты


In [5]:
# Ворую Вашу функцию визуализации, только немножечко изменю
def plot_test_vs_predictions(data, true_col, pred_col_1, pred_col_2):
    true_line = Scatter(y = data[true_col],
                       name = 'True data points')
    pred_line_1 = Scatter(y = data[pred_col_1],
                       name = 'Predictions1')
    pred_line_2 = Scatter(y = data[pred_col_2],
                       name = 'Predictions2')
    
    data = [true_line, pred_line_1, pred_line_2]
    
    iplot(Figure(data = data))
               

In [6]:
def naive_model_compare(df, resampling, function_for_agrigation): 

    if function_for_agrigation == 'sum':    # знаю, что можно написать это элегантнее, но не было времени искать, если честно 
        temp_df = df.resample(resampling).sum() 
    elif function_for_agrigation == 'mean':
        temp_df = df.resample(resampling).mean()
    elif function_for_agrigation == 'max':
        temp_df = df.resample(resampling).max()
  
    temp_df["lag_1"] = temp_df["cnt"].shift(-1)
    temp_df["lag_2"] = temp_df["cnt"].shift(-2)
    temp_df = temp_df.dropna()
    
    if resampling=='D': 
        period = 31 
    elif resampling=='W':
        period = 20
    elif resampling=='M':
        period = 6 

    train, test = temp_df[0:-period], temp_df[-period:]
    plot_test_vs_predictions(test, 'cnt', 'lag_1', 'lag_2')
    
    mae1 = np.round(mean_absolute_error(temp_df['cnt'], temp_df['lag_1']),2)
    mae2= np.round(mean_absolute_error(temp_df['cnt'], temp_df['lag_2']),2)
    arggs_now = pd.DataFrame([[resampling, function_for_agrigation, mae1, mae2]], columns=['Агрегация', 'Функция агрегации', 'MAE, lag_1', 'MAE for lag_2'])
    return arggs_now # возвращаю параметры для сравнения каждой комбинации

In [7]:
function_for_agrigation = ('sum', 'mean', 'max')
resampling = ('D','W','M') # года нет, так как данніх всего за 2 года - тестировать не на чем будет, или малоинформативно. 


for i in resampling:
    for j in function_for_agrigation:
        print(f'Период агрегации {i}, функция агрегации - {j}')
        eadvcdsv = naive_model_compare(df, i, j)
        result_table = pd.concat((result_table, naive_model_compare(df, i, j)))
        print("="*25)

Период агрегации D, функция агрегации - sum


Период агрегации D, функция агрегации - mean


Период агрегации D, функция агрегации - max


Период агрегации W, функция агрегации - sum


Период агрегации W, функция агрегации - mean


Период агрегации W, функция агрегации - max


Период агрегации M, функция агрегации - sum


Период агрегации M, функция агрегации - mean


Период агрегации M, функция агрегации - max


In [8]:
result_table.sort_values('MAE, lag_1', ignore_index=True, inplace = True)
result_table

,Агрегация,Функция агрегации,"MAE, lag_1",MAE for lag_2
0,W,mean,19.27,23.5
1,M,mean,23.55,43.47
2,D,mean,30.4,37.64
3,W,max,53.76,62.97
4,M,max,76.27,114.18
5,D,max,91.1,127.76
6,D,sum,729.6,914.0
7,W,sum,3461.92,4326.72
8,M,sum,16303.23,32042.86


Ну и смотрим на табличку/
видно, что лучший вариант - это использование функции mean с агригацией по неделям(но по месяцам или дням - тоже норм). А вот сумму использовать не стоит, особенно по дням.   
Ну и хорошо видно, что для наивных моделей чем больше лаг, тем больше ошибка - они пригодны только для ближайшего прогнозирования. 